<a href="https://colab.research.google.com/github/thucth-qt/DL_libraries_comparison/blob/master/Cifar10_keras_tensorflow_backend.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
load_from = 'drive' #'kaggle
if load_from =='drive':
  from google.colab import drive
  drive.mount('/content/gdrive')
  #load from drive
  !mkdir '/content/cifar' 
  !cp '/content/gdrive/Shared drives/TLK/cifar10/cifar-10.zip' -d '/content/cifar'  
elif load_from=='kaggle':
  #upload the credentials of the kaggle account from kaggle 
  from google.colab import files
  files.upload()

  #before importing the dataset we want to use this code
  !mkdir -p ~/.kaggle/
  # !rm -f ~/.kaggle/kaggle.json
  # !rm -f /content/kaggle.json
  !cp kaggle.json ~/.kaggle

  #this permissions change avoids a warning on Kaggle tool startup
  !chmod 600 ~/.kaggle/kaggle.json
  #upgrade Kaggle
  !pip uninstall -y kaggle
  !pip install --upgrade pip
  !pip install kaggle==1.5.6
  !kaggle -v
  #! kaggle datasets list

  #import the dataset we want to use for our project
  !mkdir '/content/cifar' 
  !kaggle competitions download -c 'cifar-10' -p '/content/cifar'


Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [ ]:
!unzip /content/cifar/cifar-10.zip -d /content/cifar/

Archive:  /content/cifar/cifar-10.zip
  inflating: /content/cifar/sampleSubmission.csv  
  inflating: /content/cifar/test.7z  
  inflating: /content/cifar/train.7z  
  inflating: /content/cifar/trainLabels.csv  


In [ ]:
!apt-get install p7zip-full
!rm -r -f '/content/cifar/train'
!mkdir '/content/cifar/train'
!p7zip -d -f -ao '/content/cifar/train.7z' -o '/content/cifar/train'

Reading package lists... Done
Building dependency tree       
Reading state information... Done
p7zip-full is already the newest version (16.02+dfsg-6).
The following package was automatically installed and is no longer required:
  libnvidia-common-440
Use 'apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 59 not upgraded.
/usr/bin/p7zip: ignoring unknown option -ao

7-Zip (a) [64] 16.02 : Copyright (c) 1999-2016 Igor Pavlov : 2016-05-21
p7zip Version 16.02 (locale=en_US.UTF-8,Utf16=on,HugeFiles=on,64 bits,2 CPUs Intel(R) Xeon(R) CPU @ 2.20GHz (406F0),ASM,AES-NI)

Scanning the drive for archives:
  0M Scan /content/cifar/                         1 file, 109723070 bytes (105 MiB)

Extracting archive: /content/cifar/train.7z
--
Path = /content/cifar/train.7z
Type = 7z
Physical Size = 109723070
Headers Size = 294768
Method = LZMA:26
Solid = +
Blocks = 1

  0%      2%      2% 774 - train/10695.png

In [ ]:
import os
print(len(os.listdir('/content/train')))

50000


In [ ]:
#Read label data
import pandas as pd 
id_label=pd.read_csv('/content/cifar/trainLabels.csv').values

#create a dictionary for matching id to label
id2label_dict ={}
for ele in id_label:
  id2label_dict[ele[0]] = ele[1]

In [ ]:
#load all images into memory
import os
import cv2
import numpy as np
from google.colab.patches import cv2_imshow
IMAGE_SIZE=32

def normalize(image:np.array):
  standard_image=image
  standard_image = cv2.resize(standard_image, (IMAGE_SIZE,IMAGE_SIZE))
  # standard_image = (standard_image-np.mean(standard_image)/np.std(standard_image))
  standard_image = standard_image/255

  return standard_image


#function return 2 lists that Input and Label
def load_image(folder):
  X = list()
  y = list()
  for i, file_name in enumerate(os.listdir(folder)):
    try:
      path_to_image = os.path.join(folder,file_name)
      image =  cv2.imread(path_to_image,1) 
      # cv2_imshow(image)
      # break
      if image is not None:
        standard_image = normalize(image)
        X.append(standard_image)
        id = int(os.path.splitext(file_name)[0])
        label=id2label_dict[id]
        y.append(label)
        if i>0 and (i+1)% 10000 == 0:
          print("[INFO] loading process: {}/{}".format(i+1, len(os.listdir(folder))))
    except Exception as e:
      print(e)
      # pass
  print("done!")
  return X,y

X_resource,y_resource = load_image(os.path.join("/content/train"))

[INFO] loading process: 10000/50000
[INFO] loading process: 20000/50000
[INFO] loading process: 30000/50000
[INFO] loading process: 40000/50000
[INFO] loading process: 50000/50000
done!


In [ ]:
from keras.utils import to_categorical

to_categorical([3],num_classes=7)

Using TensorFlow backend.


array([[0., 0., 0., 1., 0., 0., 0.]], dtype=float32)

In [ ]:
# from sklearn import preprocessing
# enc = preprocessing.OneHotEncoder(handle_unknown='ignore', sparse=False)
# X = [['Male'], ['Female'], ['boy'], ['girl']]
# enc.fit(X)
# print(np.array(enc.transform([['boy'],['girl']])))

In [ ]:
from sklearn import preprocessing
import numpy as np
le = preprocessing.OneHotEncoder(handle_unknown='ignore', sparse=False)
y_resource_array = np.array(y_resource).reshape(-1,1)
le.fit(y_resource_array)
X_train=np.array(X_resource)
y_train = le.transform(y_resource_array)


In [ ]:
print(X_train.shape, y_train.shape)

(50000, 32, 32, 3) (50000, 10)


In [ ]:
print(le.classes_)
from google.colab.patches import cv2_imshow
cv2_imshow(X_train[100])
print( y_train[100])

In [ ]:
#some constants 
SIZE_CHANGE_COIFFICIENT=2
NUM_OF_FILTERS=16
KERNEL_SIZE = 3
BATCH_SIZE =32
NUM_OF_EPOCHS = 100
POOL_SIZE=2

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import *
from tensorflow.keras.metrics import AUC, CategoricalAccuracy
from tensorflow.keras.callbacks import EarlyStopping,ModelCheckpoint
from tensorflow.keras.losses import CategoricalCrossentropy,Reduction

In [ ]:
#transfer learning rate
import math
def define_deep_model():

  model = Sequential()
  model.add(Conv2D( NUM_OF_FILTERS,
                    (KERNEL_SIZE,KERNEL_SIZE),
                    input_shape=(IMAGE_SIZE,IMAGE_SIZE,3),
                    kernel_initializer = 'he_uniform',
                    activation='relu',
                    padding='same'))
  # model.add(BatchNormalization(axis=-1))
  # model.add(MaxPooling2D((2,2)))
  

  model.add(Conv2D( NUM_OF_FILTERS*2,
                  (KERNEL_SIZE,KERNEL_SIZE),
                  kernel_initializer = 'he_uniform',
                  padding='same'))
  model.add(BatchNormalization(axis=-1))
  model.add(Activation(activation='relu'))
  model.add(Dropout(0.2))

  model.add(Conv2D( NUM_OF_FILTERS*4,
                (KERNEL_SIZE,KERNEL_SIZE),
                kernel_initializer = 'he_uniform',
                padding='same'))
  
  model.add(BatchNormalization(axis=-1))
  model.add(Activation(activation='relu'))
  model.add(Dropout(0.2))

  img_size = IMAGE_SIZE
  mul_coef=4
  multiplier=2
  # for i in range(int(1/2*math.log(IMAGE_SIZE,POOL_SIZE))):
  #   mul_coef*=multiplier
    model.add(Conv2D( NUM_OF_FILTERS*mul_coef,
                    (KERNEL_SIZE,KERNEL_SIZE),
                    kernel_initializer = 'he_uniform',
                    padding='same'))
    model.add(Activation(activation='relu'))

    mul_coef*=multiplier
    model.add(Conv2D( NUM_OF_FILTERS*mul_coef,
                    (KERNEL_SIZE,KERNEL_SIZE),
                    kernel_initializer = 'he_uniform',
                    padding='same'))

    model.add(BatchNormalization(axis=-1))
    model.add(Activation(activation='relu'))
    
    if img_size<POOL_SIZE:
      print('image size is smaller pooling size')
      return 
    model.add(MaxPooling2D((POOL_SIZE,POOL_SIZE)))
    img_size=img_size//POOL_SIZE
    model.add(Dropout(0.3))
  
  if img_size:
    model.add(MaxPooling2D((img_size,img_size)))

  model.add(Flatten())
  model.add(Dense(units = 64, kernel_initializer='he_uniform', activation='relu'))
  model.add(Dropout(0.5))
  model.add(Dense(units = 10, kernel_initializer='he_uniform', activation='softmax'))
  cce=CategoricalCrossentropy(reduction=Reduction.SUM)
  # cce=CategoricalCrossentropy()
  model.compile(optimizer = 'adam', loss=cce, metrics = [CategoricalAccuracy(),AUC()])
  return model

In [ ]:
import math
def define_model():
  
  model = Sequential()

  model.add(Conv2D( NUM_OF_FILTERS,
                    (KERNEL_SIZE,KERNEL_SIZE),
                    input_shape=(IMAGE_SIZE,IMAGE_SIZE,3),
                    kernel_initializer = 'he_uniform',
                    activation='relu',
                    padding='same'))
  model.add(Dropout(0.2))
  model.add(Conv2D( NUM_OF_FILTERS*2,
                (KERNEL_SIZE,KERNEL_SIZE),
                kernel_initializer = 'he_uniform',
                padding='same'))
  
  model.add(Dropout(0.2))
  model.add(BatchNormalization(axis=-1))  
  model.add(Activation(activation='relu'))
  model.add(MaxPooling2D((2,2)))


  model.add(Conv2D( NUM_OF_FILTERS*4,
                (KERNEL_SIZE,KERNEL_SIZE),
                kernel_initializer = 'he_uniform',
                padding='same'))
  
  model.add(Dropout(0.2))
  model.add(BatchNormalization(axis=-1))  
  model.add(Activation(activation='relu'))
  model.add(MaxPooling2D((2,2)))


  model.add(Conv2D( NUM_OF_FILTERS*8,
                (KERNEL_SIZE,KERNEL_SIZE),
                kernel_initializer = 'he_uniform',
                padding='same'))
  model.add(Dropout(0.2))
  model.add(Conv2D( NUM_OF_FILTERS*8,
                (KERNEL_SIZE,KERNEL_SIZE),
                kernel_initializer = 'he_uniform',
                padding='same'))
  
  model.add(Dropout(0.2))
  model.add(BatchNormalization(axis=-1))  
  model.add(Activation(activation='relu'))
  model.add(MaxPooling2D((2,2)))

  model.add(Conv2D( NUM_OF_FILTERS*16,
                (KERNEL_SIZE,KERNEL_SIZE),
                kernel_initializer = 'he_uniform',
                padding='same'))
  
  model.add(Dropout(0.2))
  model.add(MaxPooling2D((2,2)))

  model.add(Conv2D( NUM_OF_FILTERS*16,
                (KERNEL_SIZE,KERNEL_SIZE),
                kernel_initializer = 'he_uniform',
                padding='same'))
  
  model.add(Dropout(0.2))
  model.add(BatchNormalization(axis=-1))  
  model.add(Activation(activation='relu'))
  model.add(MaxPooling2D((2,2)))

  #flatten
  model.add(Flatten())
  model.add(Dropout(0.5))
  model.add(Dense(units = 128, kernel_initializer='he_uniform', activation='relu'))
  model.add(Dropout(0.5))
  model.add(Dense(units = 10, kernel_initializer='he_uniform', activation='softmax'))
  cce=CategoricalCrossentropy(reduction=Reduction.SUM)
  # cce=CategoricalCrossentropy()
  model.compile(optimizer = 'adam', loss=cce, metrics = [CategoricalAccuracy(),AUC()])
  return model

In [ ]:
mymodel = define_model()
mymodel.summary()

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_35 (Conv2D)           (None, 32, 32, 16)        448       
_________________________________________________________________
dropout_45 (Dropout)         (None, 32, 32, 16)        0         
_________________________________________________________________
conv2d_36 (Conv2D)           (None, 32, 32, 32)        4640      
_________________________________________________________________
dropout_46 (Dropout)         (None, 32, 32, 32)        0         
_________________________________________________________________
batch_normalization_20 (Batc (None, 32, 32, 32)        128       
_________________________________________________________________
activation_20 (Activation)   (None, 32, 32, 32)        0         
_________________________________________________________________
max_pooling2d_25 (MaxPooling (None, 16, 16, 32)       

In [ ]:
es = EarlyStopping(monitor='val_categorical_accuracy', mode='max', verbose=1, patience=7)
checkpoint_filepath = '/content/gdrive/Shared drives/TLK/cifar10/saved_models/checkpointKerasSet.h5'
model_checkpoint_callback = ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=False,
    monitor='val_loss',
    mode='min',
    save_best_only=True)
history = mymodel.fit(trainx, trainy, BATCH_SIZE, NUM_OF_EPOCHS,verbose=1,callbacks=[es,model_checkpoint_callback],validation_split=0.2,shuffle=True)

Epoch 1/100
1250/1250 [==============================] - 14s 11ms/step - loss: 65.7069 - categorical_accuracy: 0.2124 - auc_5: 0.7257 - val_loss: 71.2991 - val_categorical_accuracy: 0.2138 - val_auc_5: 0.6949
Epoch 2/100
1250/1250 [==============================] - 13s 11ms/step - loss: 53.1142 - categorical_accuracy: 0.3615 - auc_5: 0.8413 - val_loss: 87.3530 - val_categorical_accuracy: 0.2169 - val_auc_5: 0.6592
Epoch 3/100
1250/1250 [==============================] - 14s 11ms/step - loss: 46.6165 - categorical_accuracy: 0.4566 - auc_5: 0.8805 - val_loss: 45.0872 - val_categorical_accuracy: 0.4919 - val_auc_5: 0.8884
Epoch 4/100
1250/1250 [==============================] - 13s 11ms/step - loss: 41.2770 - categorical_accuracy: 0.5343 - auc_5: 0.9066 - val_loss: 47.4646 - val_categorical_accuracy: 0.5049 - val_auc_5: 0.8741
Epoch 5/100
1250/1250 [==============================] - 13s 11ms/step - loss: 37.4186 - categorical_accuracy: 0.5930 - auc_5: 0.9229 - val_loss: 46.9278 - val_cate

In [ ]:
#save model
NAME='checkpoint.h5'

mymodel.save('/content/'+NAME)
# !mkdir '/content/gdrive/Shared drives/TLK/cifar10/saved_models'
source = '/content/'+NAME
!cp  $source -d '/content/gdrive/Shared drives/TLK/cifar10/saved_models'

In [ ]:
#visualize accuracy
from matplotlib import pyplot as plt
plt.plot(history.history['categorical_accuracy'])
plt.plot(history.history['val_categorical_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()


NameError: ignored

In [ ]:
#visualize loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()

NameError: ignored

In [ ]:
print(le.categories_)

[array(['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog',
       'horse', 'ship', 'truck'], dtype='<U10')]


In [ ]:
print(type(testy[0,0]))

<class 'numpy.uint8'>


In [ ]:
!cp '/content/gdrive/Shared drives/TLK/cifar10/saved_models/checkpointmorning.h5' -d '/content/cifar/'

In [ ]:
from tensorflow.keras.models import load_model
mymodel= load_model('/content/cifar/checkpointmorning.h5')

In [ ]:
from tensorflow.keras.datasets import cifar10
def ordinal2str(num):
  name=['airplane','automobile','bird','cat','deer','dog','frog','horse','ship','truck']
  return name[num]
(trainx, trainy),(testx,testy)=cifar10.load_data()
trainy=le.transform(np.vectorize(ordinal2str)(trainy))
print(testx.shape)
print(testy.shape)

(10000, 32, 32, 3)
(10000, 1)
313/313 [==============================] - 1s 5ms/step - loss: 20.6492 - categorical_accuracy: 0.7978 - auc_4: 0.9726


[20.64923667907715, 0.7978000044822693, 0.9726195931434631]

In [ ]:
mymodel.evaluate(testx,le.transform(np.vectorize(ordinal2str)(testy)))

313/313 [==============================] - 1s 5ms/step - loss: 17.7367 - categorical_accuracy: 0.8216 - auc_5: 0.9799


[17.73672866821289, 0.8216000199317932, 0.9799376130104065]

In [ ]:
from google.colab.patches import cv2_imshow
from tensorflow.keras.datasets import cifar10
def ordinal2str(num):
  name=['airplane','automobile','bird','cat','deer','dog','frog','horse','ship','truck']
  return name[num]
_,(testx,testy)=cifar10.load_data()
print(testx.shape)
print(testy.shape)

count=0
for i, input_image in enumerate(list(list(testx))):
  test_input= np.array(normalize(input_image)).reshape((1,IMAGE_SIZE,IMAGE_SIZE,-1))
  # print(test_input.shape)
  output= mymodel.predict(test_input)
  name=ordinal2str(testy[i][0])
  true_label = le.transform([[name]])
  # print(true_label)
  # true_label=y_eval[i]
  if np.argmax(output) == np.argmax(true_label):
    count+=1
    # print('[INFO] predict correctly {}/{} images'.format(count,i+1))
  else:
    # print('[FAIL] output: {} | true label: {}'.format(output,true_label))
    # cv2_imshow(input_image)
    pass
    # pass
print(count/i)
# for i in range(100):
#   if testy[i]==8:
#     print("----")
#     cv2_imshow(testx[i])
#     print(testy[i])
#     print("----")





(10000, 32, 32, 3)
(10000, 1)
0.7027702770277028


In [ ]:
!nvidia-smi

Sat Jun 20 00:28:06 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.36.06    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   73C    P0    25W /  75W |   7353MiB /  7611MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------